# AMSC698K Homework 10
##### Elijah Kin & Noorain Noorani

In [ ]:
# TODO

### Problem A
1. The Hadamard Test circuit takes a qubit $q_0$, prepared as $|+\rangle$, and state $|\psi\rangle \in \mathbb{C}^n$ and applies to it the controlled unitary CU. Describe how this test works and what the measurement of $q_0$ tells us about $U |\psi\rangle$. 

2. In the VQLS algorithm the Hadamard Test is applied in a modified form since two controlled unitaries are provided for the test. Describe how this test results in an "overlap probability" (or cosine similarity, a central measure in NLP).

In [ ]:
# TODO

### Problem B
1. Suppose a LSE, decomposed into quantum gates, is given by:
$$ A = \mathbb{I} + 0.2 X_0 Z_1 + 0.2 X_0; \quad |b\rangle = H_0 H_1 H_2 |0\rangle $$
Apply the VQLS algorithm and compare with classical solutions using $\texttt{scipy.linalg.solve()}$ and $\texttt{scipy.linalg.inv()}$.

2. (extra credit - 30pts) Solve this problem using the HHL algorithm.

In [ ]:
# TODO

### Problem C
Given the inhomogeneous linear system:
$$ \begin{align*} x_1 - x_2 + x_3 - x_4 &= 2 \\ x_1 - x_2 + x_3 + x_4 &= 0 \\ 4x_1 - 4x_2 + 4x_3 &= 4 \\ 2x_1 - 2x_2 + 2x_3 - x_4 &= 3 \end{align*} $$
Write this LSE in matrix form and decompose the coefficient matrix $A$ and vector $b$ to apply the VQLS algorithm and compare its result with the "classical" solutions.

In [ ]:
# TODO